# EPL Player Props Betting Model: Shots & Shots on Target

A probabilistic model for finding edges in EPL player shot markets.

## Overview

This notebook implements:
1. **Two-stage minutes prediction** (start probability + conditional minutes)
2. **Hierarchical Negative Binomial model** for shots/SOT with player & opponent effects
3. **De-vigging** bookmaker odds to extract fair probabilities
4. **Walk-forward backtesting** with CLV as primary edge metric
5. **Weekly candidate generation** with Kelly-based stake sizing

### Assumptions & Limitations

- No guaranteed profit is claimed
- Edges must be validated via CLV and calibration over large sample sizes
- Model assumes pre-match betting only (no in-play)
- Requires 1000+ bets for statistical significance
- Follow local gambling laws and sportsbook terms of service

## 1. Setup & Configuration

In [ ]:
# Core dependencies
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Statistical modeling
from scipy import stats
from scipy.optimize import brentq, minimize
from scipy.special import gammaln
import statsmodels.api as sm
from statsmodels.discrete.count_model import NegativeBinomialP

# Machine learning
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import brier_score_loss, log_loss, roc_auc_score
from sklearn.calibration import calibration_curve

# Visualization
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
plt.style.use('seaborn-v0_8-whitegrid')

# Set random seed for reproducibility
np.random.seed(42)

print("Dependencies loaded successfully")

In [ ]:
# Configuration
CONFIG = {
    # Data settings
    'seasons': ['2019-20', '2020-21', '2021-22', '2022-23', '2023-24'],
    'min_minutes_for_model': 10,  # Minimum minutes to include in training
    
    # Feature engineering
    'ewma_spans': [3, 10, 20],  # Short, medium, long-term EWMA windows
    'rolling_windows': [5, 10],  # Rolling average windows
    
    # Minutes model
    'congestion_days_threshold': 4,  # Days between matches for rotation risk
    
    # Shot model
    'min_player_appearances': 10,  # Minimum games for player effect
    
    # Betting rules
    'min_ev_threshold': 0.03,  # Minimum 3% EV to bet
    'kelly_fraction': 0.25,   # Quarter Kelly
    'max_bet_fraction': 0.02,  # Max 2% of bankroll per bet
    'max_bets_per_match': 3,   # Limit correlated bets
    
    # Backtesting
    'train_seasons': 3,  # Minimum seasons for training
    'test_window_weeks': 4,  # Test on 4 weeks before retraining
}

print("Configuration set")

## 2. Data Loading & Schema

This section handles data ingestion. For real usage, connect to FBref via `soccerdata`.
Here we generate realistic synthetic data matching the expected schema.

In [ ]:
def generate_synthetic_epl_data(n_seasons=5, n_teams=20, players_per_team=25):
    """
    Generate realistic synthetic EPL player-match data for model development.
    Replace with actual FBref data loading for production use.
    """
    np.random.seed(42)
    
    # Generate teams
    teams = [f'Team_{i}' for i in range(n_teams)]
    
    # Generate players with inherent shot tendencies
    players = []
    player_attributes = {}
    
    positions = ['GK', 'CB', 'FB', 'DM', 'CM', 'AM', 'W', 'ST']
    position_shot_rates = {'GK': 0.01, 'CB': 0.3, 'FB': 0.5, 'DM': 0.8, 
                           'CM': 1.2, 'AM': 2.0, 'W': 2.2, 'ST': 3.5}
    position_start_prob = {'GK': 0.95, 'CB': 0.85, 'FB': 0.80, 'DM': 0.75,
                           'CM': 0.70, 'AM': 0.65, 'W': 0.60, 'ST': 0.70}
    
    player_id = 0
    for team in teams:
        for _ in range(players_per_team):
            pos = np.random.choice(positions, p=[0.08, 0.16, 0.16, 0.12, 0.16, 0.12, 0.12, 0.08])
            base_rate = position_shot_rates[pos]
            # Add player-specific variation
            player_shot_rate = base_rate * np.random.lognormal(0, 0.3)
            player_start_base = position_start_prob[pos] * np.random.uniform(0.7, 1.1)
            
            player_attributes[player_id] = {
                'team': team,
                'position': pos,
                'base_shot_rate': player_shot_rate,
                'base_start_prob': min(0.95, player_start_base),
                'sot_conversion': np.random.beta(3, 4)  # Shots on target / shots
            }
            players.append(player_id)
            player_id += 1
    
    # Generate team attack/defense strengths
    team_attack = {t: np.random.normal(1.0, 0.25) for t in teams}
    team_defense = {t: np.random.normal(1.0, 0.25) for t in teams}
    
    # Generate match-level data
    records = []
    match_id = 0
    
    for season_idx in range(n_seasons):
        season = CONFIG['seasons'][season_idx] if season_idx < len(CONFIG['seasons']) else f'Season_{season_idx}'
        
        # 38 matchweeks per season
        for matchweek in range(1, 39):
            match_date = pd.Timestamp(f'202{season_idx}-08-01') + pd.Timedelta(weeks=matchweek-1)
            
            # Generate 10 matches per matchweek
            shuffled_teams = np.random.permutation(teams)
            for i in range(0, n_teams, 2):
                home_team = shuffled_teams[i]
                away_team = shuffled_teams[i+1]
                
                # Match context
                home_advantage = 1.15
                match_tempo = np.random.uniform(0.8, 1.2)  # Some matches more open
                
                # Days since last match (congestion)
                days_rest_home = np.random.choice([3, 4, 5, 6, 7, 8], p=[0.1, 0.2, 0.15, 0.15, 0.3, 0.1])
                days_rest_away = np.random.choice([3, 4, 5, 6, 7, 8], p=[0.1, 0.2, 0.15, 0.15, 0.3, 0.1])
                
                # Generate player records for both teams
                for is_home, team, opp_team, days_rest in [(True, home_team, away_team, days_rest_home),
                                                            (False, away_team, home_team, days_rest_away)]:
                    team_players = [p for p in players if player_attributes[p]['team'] == team]
                    
                    for player in team_players:
                        attrs = player_attributes[player]
                        
                        # Determine if player starts (rotation based on congestion)
                        congestion_factor = 0.9 if days_rest < CONFIG['congestion_days_threshold'] else 1.0
                        start_prob = attrs['base_start_prob'] * congestion_factor
                        started = np.random.random() < start_prob
                        
                        # Minutes played
                        if started:
                            # Starters: mostly full match, some subbed off
                            if np.random.random() < 0.75:
                                minutes = np.random.randint(85, 91)
                            else:
                                minutes = np.random.randint(45, 85)
                        else:
                            # Subs: many don't play, some get short appearances
                            if np.random.random() < 0.6:
                                minutes = 0
                            else:
                                minutes = np.random.randint(1, 45)
                        
                        if minutes == 0:
                            shots, sot, goals, xg = 0, 0, 0, 0.0
                        else:
                            # Shot generation based on rate, minutes, context
                            rate_per90 = attrs['base_shot_rate']
                            rate_per90 *= team_attack[team]  # Team attacking strength
                            rate_per90 *= (2 - team_defense[opp_team])  # Opponent defensive weakness
                            rate_per90 *= match_tempo
                            if is_home:
                                rate_per90 *= home_advantage
                            
                            # Draw from Negative Binomial (overdispersed)
                            expected_shots = rate_per90 * (minutes / 90)
                            dispersion = 2.0  # Overdispersion parameter
                            if expected_shots > 0:
                                r = expected_shots / (dispersion - 1) if dispersion > 1 else expected_shots
                                p = 1 / dispersion
                                shots = stats.nbinom.rvs(n=max(0.1, r), p=min(0.99, max(0.01, p)))
                            else:
                                shots = 0
                            
                            # Shots on target
                            sot = np.random.binomial(shots, attrs['sot_conversion'])
                            
                            # Goals (roughly 10% of shots)
                            goals = np.random.binomial(sot, 0.35)
                            
                            # xG (correlated with shots but not perfectly)
                            xg = shots * np.random.uniform(0.08, 0.15) if shots > 0 else 0
                        
                        records.append({
                            'match_id': match_id,
                            'season': season,
                            'matchweek': matchweek,
                            'date': match_date,
                            'player_id': player,
                            'player_name': f'Player_{player}',
                            'team': team,
                            'opponent': opp_team,
                            'is_home': is_home,
                            'position': attrs['position'],
                            'started': started,
                            'minutes': minutes,
                            'shots': shots,
                            'shots_on_target': sot,
                            'goals': goals,
                            'xg': round(xg, 3),
                            'days_rest': days_rest,
                        })
                
                match_id += 1
    
    df = pd.DataFrame(records)
    df['date'] = pd.to_datetime(df['date'])
    df = df.sort_values(['date', 'match_id', 'player_id']).reset_index(drop=True)
    
    return df, player_attributes, team_attack, team_defense

# Generate data
print("Generating synthetic EPL data...")
df_raw, player_attrs, team_atk, team_def = generate_synthetic_epl_data()
print(f"Generated {len(df_raw):,} player-match records")
print(f"Date range: {df_raw['date'].min()} to {df_raw['date'].max()}")
print(f"Unique players: {df_raw['player_id'].nunique()}")
print(f"Unique matches: {df_raw['match_id'].nunique()}")

In [ ]:
# Data schema documentation
print("\n=== DATA SCHEMA ===")
print(df_raw.dtypes)
print("\n=== SAMPLE DATA ===")
df_raw.head(10)

In [ ]:
# Real data loading function (uncomment to use with actual FBref data)
"""
def load_fbref_data(seasons):
    import soccerdata as sd
    
    fbref = sd.FBref('ENG-Premier League', seasons)
    
    # Load player match stats
    shooting = fbref.read_player_match_stats(stat_type='shooting')
    summary = fbref.read_player_match_stats(stat_type='summary')
    
    # Merge and clean
    df = shooting.merge(summary[['minutes', 'started']], 
                        left_index=True, right_index=True, how='left')
    
    # Reset index and clean column names
    df = df.reset_index()
    df.columns = [c.lower().replace(' ', '_') for c in df.columns]
    
    return df

# df_raw = load_fbref_data(['2324', '2223', '2122', '2021', '1920'])
"""
print("FBref loading function defined (commented out - using synthetic data)")

## 3. Data Cleaning & Feature Engineering

In [ ]:
def create_features(df):
    """
    Create all features for minutes and shot models.
    Uses only backward-looking information to prevent leakage.
    """
    df = df.copy()
    df = df.sort_values(['player_id', 'date']).reset_index(drop=True)
    
    # === Player-level rolling features ===
    for span in CONFIG['ewma_spans']:
        # EWMA for shot rates (shifted to prevent leakage)
        df[f'shots_ewma_{span}'] = df.groupby('player_id')['shots'].transform(
            lambda x: x.shift(1).ewm(span=span, min_periods=1).mean()
        )
        df[f'sot_ewma_{span}'] = df.groupby('player_id')['shots_on_target'].transform(
            lambda x: x.shift(1).ewm(span=span, min_periods=1).mean()
        )
        df[f'minutes_ewma_{span}'] = df.groupby('player_id')['minutes'].transform(
            lambda x: x.shift(1).ewm(span=span, min_periods=1).mean()
        )
        df[f'xg_ewma_{span}'] = df.groupby('player_id')['xg'].transform(
            lambda x: x.shift(1).ewm(span=span, min_periods=1).mean()
        )
    
    # Rolling start probability
    df['starts_rolling_5'] = df.groupby('player_id')['started'].transform(
        lambda x: x.shift(1).rolling(5, min_periods=1).mean()
    )
    df['starts_rolling_10'] = df.groupby('player_id')['started'].transform(
        lambda x: x.shift(1).rolling(10, min_periods=1).mean()
    )
    
    # Per-90 rates (using EWMA minutes to avoid division by current minutes)
    df['shots_per90_ewma'] = np.where(
        df['minutes_ewma_10'] > 0,
        df['shots_ewma_10'] / (df['minutes_ewma_10'] / 90),
        0
    )
    df['sot_per90_ewma'] = np.where(
        df['minutes_ewma_10'] > 0,
        df['sot_ewma_10'] / (df['minutes_ewma_10'] / 90),
        0
    )
    
    # === Team-level features ===
    # Team's rolling shot volume (proxy for attacking style)
    team_shots = df.groupby(['team', 'date'])['shots'].sum().reset_index()
    team_shots = team_shots.sort_values(['team', 'date'])
    team_shots['team_shots_ewma'] = team_shots.groupby('team')['shots'].transform(
        lambda x: x.shift(1).ewm(span=10, min_periods=1).mean()
    )
    team_shots_dict = team_shots.set_index(['team', 'date'])['team_shots_ewma'].to_dict()
    df['team_shots_ewma'] = df.apply(
        lambda r: team_shots_dict.get((r['team'], r['date']), np.nan), axis=1
    )
    
    # === Opponent-level features ===
    # Opponent's shots conceded (defensive quality)
    opp_shots = df.groupby(['opponent', 'date'])['shots'].sum().reset_index()
    opp_shots = opp_shots.sort_values(['opponent', 'date'])
    opp_shots['opp_shots_conceded_ewma'] = opp_shots.groupby('opponent')['shots'].transform(
        lambda x: x.shift(1).ewm(span=10, min_periods=1).mean()
    )
    opp_shots_dict = opp_shots.set_index(['opponent', 'date'])['opp_shots_conceded_ewma'].to_dict()
    df['opp_shots_conceded_ewma'] = df.apply(
        lambda r: opp_shots_dict.get((r['opponent'], r['date']), np.nan), axis=1
    )
    
    # === Match context features ===
    df['is_home_int'] = df['is_home'].astype(int)
    df['congestion'] = (df['days_rest'] < CONFIG['congestion_days_threshold']).astype(int)
    
    # Position encoding
    position_map = {'GK': 0, 'CB': 1, 'FB': 2, 'DM': 3, 'CM': 4, 'AM': 5, 'W': 6, 'ST': 7}
    df['position_code'] = df['position'].map(position_map)
    df['is_attacker'] = df['position'].isin(['AM', 'W', 'ST']).astype(int)
    
    # Player appearance count (for filtering)
    df['player_game_num'] = df.groupby('player_id').cumcount() + 1
    
    # Fill NaN with position/league medians where appropriate
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].median())
    
    return df

print("Creating features...")
df = create_features(df_raw)
print(f"Features created. Shape: {df.shape}")
print(f"\nNew columns: {[c for c in df.columns if c not in df_raw.columns]}")

## 4. Exploratory Data Analysis

In [ ]:
# Shot distribution analysis
fig, axes = plt.subplots(2, 3, figsize=(14, 8))

# Filter to players with meaningful minutes
df_played = df[df['minutes'] >= CONFIG['min_minutes_for_model']]

# Shot distribution (all players who played)
axes[0, 0].hist(df_played['shots'], bins=range(0, 12), edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('Shots')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title(f'Shot Distribution (n={len(df_played):,})')
axes[0, 0].axvline(df_played['shots'].mean(), color='red', linestyle='--', label=f'Mean: {df_played["shots"].mean():.2f}')
axes[0, 0].legend()

# SOT distribution
axes[0, 1].hist(df_played['shots_on_target'], bins=range(0, 8), edgecolor='black', alpha=0.7, color='green')
axes[0, 1].set_xlabel('Shots on Target')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title(f'SOT Distribution')
axes[0, 1].axvline(df_played['shots_on_target'].mean(), color='red', linestyle='--', label=f'Mean: {df_played["shots_on_target"].mean():.2f}')
axes[0, 1].legend()

# Minutes distribution (bimodal check)
axes[0, 2].hist(df['minutes'], bins=range(0, 95, 5), edgecolor='black', alpha=0.7, color='orange')
axes[0, 2].set_xlabel('Minutes')
axes[0, 2].set_ylabel('Frequency')
axes[0, 2].set_title('Minutes Distribution (Bimodal)')

# Shots by position
position_order = ['GK', 'CB', 'FB', 'DM', 'CM', 'AM', 'W', 'ST']
shots_by_pos = df_played.groupby('position')['shots'].mean().reindex(position_order)
axes[1, 0].bar(shots_by_pos.index, shots_by_pos.values, color='steelblue', edgecolor='black')
axes[1, 0].set_xlabel('Position')
axes[1, 0].set_ylabel('Mean Shots')
axes[1, 0].set_title('Shots by Position')
axes[1, 0].tick_params(axis='x', rotation=45)

# Variance vs Mean (overdispersion check)
player_stats = df_played.groupby('player_id').agg({'shots': ['mean', 'var']}).dropna()
player_stats.columns = ['mean', 'var']
player_stats = player_stats[player_stats['mean'] > 0]
axes[1, 1].scatter(player_stats['mean'], player_stats['var'], alpha=0.3)
max_val = max(player_stats['mean'].max(), player_stats['var'].max())
axes[1, 1].plot([0, max_val], [0, max_val], 'r--', label='Var = Mean (Poisson)')
axes[1, 1].set_xlabel('Mean Shots')
axes[1, 1].set_ylabel('Variance')
axes[1, 1].set_title('Overdispersion Check')
axes[1, 1].legend()

# Home vs Away
home_away = df_played.groupby('is_home')['shots'].mean()
axes[1, 2].bar(['Away', 'Home'], home_away.values, color=['coral', 'seagreen'], edgecolor='black')
axes[1, 2].set_ylabel('Mean Shots')
axes[1, 2].set_title('Home vs Away Shots')

plt.tight_layout()
plt.savefig('eda_shots.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nOverdispersion ratio (var/mean): {df_played['shots'].var() / df_played['shots'].mean():.2f}")
print("Values > 1 indicate overdispersion, supporting Negative Binomial over Poisson")

## 5. Minutes Prediction Model (Two-Stage)

Critical for accurate shot predictions: we need to model minutes uncertainty.

In [ ]:
class MinutesModel:
    """
    Two-stage minutes prediction:
    1. P(start) - Classification
    2. Minutes | start vs Minutes | sub - Conditional distributions
    """
    
    def __init__(self):
        self.start_model = GradientBoostingClassifier(
            n_estimators=100, max_depth=4, learning_rate=0.1, random_state=42
        )
        self.scaler = StandardScaler()
        
        # Distribution parameters (fit from data)
        self.starter_mu = 82
        self.starter_sigma = 12
        self.sub_mu = 15
        self.sub_sigma = 12
        
        self.feature_cols = [
            'starts_rolling_5', 'starts_rolling_10',
            'minutes_ewma_3', 'minutes_ewma_10',
            'congestion', 'position_code', 'is_attacker'
        ]
    
    def fit(self, df):
        """Fit start probability model and distribution parameters."""
        # Filter to valid training data
        train_df = df[df['player_game_num'] > 3].copy()  # Need history
        
        X = train_df[self.feature_cols].values
        y = train_df['started'].values
        
        X_scaled = self.scaler.fit_transform(X)
        self.start_model.fit(X_scaled, y)
        
        # Fit conditional distributions from data
        starters = train_df[train_df['started'] == True]
        subs = train_df[(train_df['started'] == False) & (train_df['minutes'] > 0)]
        
        self.starter_mu = starters['minutes'].mean()
        self.starter_sigma = starters['minutes'].std()
        self.sub_mu = subs['minutes'].mean() if len(subs) > 0 else 15
        self.sub_sigma = subs['minutes'].std() if len(subs) > 0 else 10
        
        return self
    
    def predict_start_prob(self, df):
        """Predict probability of starting."""
        X = df[self.feature_cols].values
        X_scaled = self.scaler.transform(X)
        return self.start_model.predict_proba(X_scaled)[:, 1]
    
    def sample_minutes(self, start_probs, n_samples=1000):
        """
        Sample from minutes distribution given start probabilities.
        Returns array of shape (n_players, n_samples).
        """
        n_players = len(start_probs)
        samples = np.zeros((n_players, n_samples))
        
        for i, p_start in enumerate(start_probs):
            starts = np.random.random(n_samples) < p_start
            
            # Sample minutes conditional on start/sub
            starter_mins = np.clip(
                np.random.normal(self.starter_mu, self.starter_sigma, n_samples),
                45, 90
            )
            sub_mins = np.clip(
                np.random.normal(self.sub_mu, self.sub_sigma, n_samples),
                0, 45
            )
            
            # Also model probability of not playing at all (if sub)
            no_play = np.random.random(n_samples) < 0.5  # 50% of subs don't play
            sub_mins = np.where(no_play, 0, sub_mins)
            
            samples[i] = np.where(starts, starter_mins, sub_mins)
        
        return samples

# Train minutes model
print("Training minutes model...")
minutes_model = MinutesModel()
minutes_model.fit(df)

# Evaluate on held-out data
test_df = df[df['season'] == CONFIG['seasons'][-1]].copy()
test_df = test_df[test_df['player_game_num'] > 3]

start_probs = minutes_model.predict_start_prob(test_df)
start_auc = roc_auc_score(test_df['started'], start_probs)
start_brier = brier_score_loss(test_df['started'], start_probs)

print(f"\nStart Probability Model Performance:")
print(f"  AUC-ROC: {start_auc:.3f}")
print(f"  Brier Score: {start_brier:.3f}")
print(f"  Starter params: μ={minutes_model.starter_mu:.1f}, σ={minutes_model.starter_sigma:.1f}")
print(f"  Sub params: μ={minutes_model.sub_mu:.1f}, σ={minutes_model.sub_sigma:.1f}")

In [ ]:
# Visualize minutes model
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Start probability calibration
prob_true, prob_pred = calibration_curve(test_df['started'], start_probs, n_bins=10)
axes[0].plot(prob_pred, prob_true, 's-', label='Model')
axes[0].plot([0, 1], [0, 1], 'k--', label='Perfect')
axes[0].set_xlabel('Predicted P(Start)')
axes[0].set_ylabel('Actual Start Rate')
axes[0].set_title('Start Probability Calibration')
axes[0].legend()

# Sample minutes distribution for a typical starter
sample_mins = minutes_model.sample_minutes(np.array([0.8]), n_samples=5000)[0]
axes[1].hist(sample_mins, bins=30, edgecolor='black', alpha=0.7)
axes[1].axvline(sample_mins.mean(), color='red', linestyle='--', label=f'Mean: {sample_mins.mean():.1f}')
axes[1].set_xlabel('Minutes')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Minutes Distribution (P(start)=0.8)')
axes[1].legend()

# Sample minutes distribution for a typical sub
sample_mins_sub = minutes_model.sample_minutes(np.array([0.3]), n_samples=5000)[0]
axes[2].hist(sample_mins_sub, bins=30, edgecolor='black', alpha=0.7, color='orange')
axes[2].axvline(sample_mins_sub.mean(), color='red', linestyle='--', label=f'Mean: {sample_mins_sub.mean():.1f}')
axes[2].set_xlabel('Minutes')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Minutes Distribution (P(start)=0.3)')
axes[2].legend()

plt.tight_layout()
plt.savefig('minutes_model.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Shot Distribution Models (Hierarchical Negative Binomial)

In [ ]:
class HierarchicalShotModel:
    """
    Negative Binomial model for shots with player and team effects.
    Uses statsmodels for frequentist estimation with random effects approximation.
    """
    
    def __init__(self, target='shots'):
        self.target = target
        self.model = None
        self.player_effects = {}
        self.team_effects = {}
        self.opponent_effects = {}
        self.dispersion = 1.0
        
        self.feature_cols = [
            'is_home_int', 'position_code', 'is_attacker',
            'shots_per90_ewma', 'team_shots_ewma', 'opp_shots_conceded_ewma'
        ]
    
    def fit(self, df):
        """Fit Negative Binomial regression with exposure (minutes)."""
        # Filter training data
        train_df = df[
            (df['minutes'] >= CONFIG['min_minutes_for_model']) &
            (df['player_game_num'] >= 5)
        ].copy()
        
        # Prepare features
        X = train_df[self.feature_cols].copy()
        X = sm.add_constant(X)
        y = train_df[self.target].values
        exposure = train_df['minutes'].values / 90  # Exposure in 90-minute units
        
        # Fit Negative Binomial
        self.model = NegativeBinomialP(
            y, X, exposure=exposure
        ).fit(disp=False, maxiter=100)
        
        self.dispersion = self.model.params[-1] if hasattr(self.model, 'params') else 1.0
        
        # Compute player-level effects (residual-based approximation)
        train_df['predicted_rate'] = self.model.predict(X) / exposure
        train_df['residual'] = (train_df[self.target] / (train_df['minutes']/90 + 0.1)) - train_df['predicted_rate']
        
        # Shrink player effects toward zero
        player_resid = train_df.groupby('player_id')['residual'].agg(['mean', 'count'])
        # Shrinkage factor based on sample size
        shrinkage = player_resid['count'] / (player_resid['count'] + 10)
        self.player_effects = (player_resid['mean'] * shrinkage).to_dict()
        
        # Team effects
        team_resid = train_df.groupby('team')['residual'].mean()
        self.team_effects = (team_resid * 0.5).to_dict()  # Shrink team effects
        
        # Opponent effects
        opp_resid = train_df.groupby('opponent')['residual'].mean()
        self.opponent_effects = (opp_resid * 0.5).to_dict()
        
        return self
    
    def predict_rate(self, df):
        """Predict shot rate per 90 minutes."""
        X = df[self.feature_cols].copy()
        X = sm.add_constant(X, has_constant='add')
        
        # Base prediction (rate per 90)
        base_rate = self.model.predict(X)
        
        # Add random effects
        player_adj = df['player_id'].map(self.player_effects).fillna(0)
        team_adj = df['team'].map(self.team_effects).fillna(0)
        opp_adj = df['opponent'].map(self.opponent_effects).fillna(0)
        
        adjusted_rate = base_rate + player_adj + team_adj + opp_adj
        return np.maximum(adjusted_rate, 0.01)  # Floor at small positive value
    
    def predict_proba_over(self, df, line, minutes_samples):
        """
        Predict P(shots > line) integrating over minutes uncertainty.
        
        Args:
            df: DataFrame with features
            line: Betting line (e.g., 1.5, 2.5)
            minutes_samples: Array of shape (n_players, n_samples)
        
        Returns:
            Array of P(over) for each player
        """
        rate_per90 = self.predict_rate(df).values
        n_players = len(df)
        n_samples = minutes_samples.shape[1]
        
        # Get dispersion parameter
        alpha = max(0.1, getattr(self, 'dispersion', 1.0))
        
        probs_over = np.zeros(n_players)
        
        for i in range(n_players):
            # For each minutes sample, compute P(shots > line)
            p_over_samples = []
            
            for mins in minutes_samples[i]:
                if mins < 5:  # Very short appearance
                    p_over_samples.append(0.0)
                    continue
                
                mu = rate_per90[i] * (mins / 90)
                
                # Negative Binomial: parameterized by mean and dispersion
                # scipy uses n, p parameterization: n = 1/alpha, p = 1/(1 + alpha*mu)
                r = 1 / alpha
                p = r / (r + mu)
                
                # P(X > line) = 1 - P(X <= floor(line))
                p_over = 1 - stats.nbinom.cdf(int(line), n=r, p=p)
                p_over_samples.append(p_over)
            
            probs_over[i] = np.mean(p_over_samples)
        
        return probs_over

# Train shot models
print("Training shot models...")

# Use earlier seasons for training
train_seasons = CONFIG['seasons'][:-1]
train_df = df[df['season'].isin(train_seasons)]

shot_model = HierarchicalShotModel(target='shots')
shot_model.fit(train_df)

sot_model = HierarchicalShotModel(target='shots_on_target')
sot_model.fit(train_df)

print(f"\nShots model coefficients:")
print(shot_model.model.summary().tables[1])
print(f"\nNumber of player effects: {len(shot_model.player_effects)}")

In [ ]:
# Evaluate shot model predictions
test_df = df[df['season'] == CONFIG['seasons'][-1]].copy()
test_df = test_df[
    (test_df['minutes'] >= CONFIG['min_minutes_for_model']) &
    (test_df['player_game_num'] >= 5)
]

# Predict rates
test_df['pred_shot_rate'] = shot_model.predict_rate(test_df)
test_df['pred_shots'] = test_df['pred_shot_rate'] * (test_df['minutes'] / 90)

# Calculate errors
mae = np.abs(test_df['shots'] - test_df['pred_shots']).mean()
rmse = np.sqrt(((test_df['shots'] - test_df['pred_shots'])**2).mean())

print(f"Shot Model Performance (using actual minutes):")
print(f"  MAE: {mae:.3f}")
print(f"  RMSE: {rmse:.3f}")
print(f"  Mean actual shots: {test_df['shots'].mean():.3f}")
print(f"  Mean predicted shots: {test_df['pred_shots'].mean():.3f}")

## 7. Odds De-Vigging & Expected Value

In [ ]:
class OddsProcessor:
    """Handle bookmaker odds: de-vigging, EV calculation, and Kelly sizing."""
    
    @staticmethod
    def american_to_decimal(american):
        """Convert American odds to decimal."""
        if american > 0:
            return (american / 100) + 1
        else:
            return (100 / abs(american)) + 1
    
    @staticmethod
    def decimal_to_implied(decimal_odds):
        """Convert decimal odds to implied probability."""
        return 1 / decimal_odds
    
    @staticmethod
    def devig_multiplicative(odds_over, odds_under):
        """
        Multiplicative (proportional) de-vig method.
        Distributes vig proportionally.
        """
        imp_over = 1 / odds_over
        imp_under = 1 / odds_under
        total = imp_over + imp_under
        
        return {
            'fair_prob_over': imp_over / total,
            'fair_prob_under': imp_under / total,
            'vig': (total - 1) * 100
        }
    
    @staticmethod
    def devig_power(odds_over, odds_under):
        """
        Power method de-vig - better for favorite/longshot bias.
        Finds k such that implied_over^k + implied_under^k = 1
        """
        imp_over = 1 / odds_over
        imp_under = 1 / odds_under
        
        def objective(k):
            return (imp_over ** k) + (imp_under ** k) - 1
        
        try:
            k = brentq(objective, 0.1, 2.0)
            fair_over = imp_over ** k
            fair_under = imp_under ** k
        except:
            # Fall back to multiplicative if power method fails
            total = imp_over + imp_under
            fair_over = imp_over / total
            fair_under = imp_under / total
            k = 1.0
        
        return {
            'fair_prob_over': fair_over,
            'fair_prob_under': fair_under,
            'k': k,
            'vig': (imp_over + imp_under - 1) * 100
        }
    
    @staticmethod
    def calculate_ev(model_prob, decimal_odds):
        """
        Calculate expected value of a bet.
        EV = P(win) * (odds - 1) - P(lose)
        """
        p_win = model_prob
        p_lose = 1 - model_prob
        ev = p_win * (decimal_odds - 1) - p_lose
        return ev
    
    @staticmethod
    def kelly_stake(model_prob, decimal_odds, fraction=0.25, max_stake=0.02):
        """
        Calculate Kelly criterion stake.
        
        Args:
            model_prob: Your estimated probability of winning
            decimal_odds: Decimal odds offered
            fraction: Kelly fraction (0.25 = quarter Kelly)
            max_stake: Maximum stake as fraction of bankroll
        
        Returns:
            Recommended stake as fraction of bankroll
        """
        b = decimal_odds - 1  # Net odds
        q = 1 - model_prob    # Probability of losing
        
        # Kelly formula: (bp - q) / b
        full_kelly = (b * model_prob - q) / b
        
        # Apply fraction and cap
        stake = max(0, full_kelly * fraction)
        stake = min(stake, max_stake)
        
        return stake

# Example: de-vig some odds
print("De-vig Example:")
print("Book offers: Over 1.5 shots @ 1.80, Under 1.5 shots @ 2.00")

result_mult = OddsProcessor.devig_multiplicative(1.80, 2.00)
result_power = OddsProcessor.devig_power(1.80, 2.00)

print(f"\nMultiplicative method:")
print(f"  Fair P(Over): {result_mult['fair_prob_over']:.3f}")
print(f"  Fair P(Under): {result_mult['fair_prob_under']:.3f}")
print(f"  Vig: {result_mult['vig']:.1f}%")

print(f"\nPower method:")
print(f"  Fair P(Over): {result_power['fair_prob_over']:.3f}")
print(f"  Fair P(Under): {result_power['fair_prob_under']:.3f}")
print(f"  Vig: {result_power['vig']:.1f}%")

In [ ]:
def generate_synthetic_odds(df, shot_model, minutes_model):
    """
    Generate synthetic bookmaker odds for backtesting.
    In production, load real odds from CSV/API.
    """
    np.random.seed(123)
    
    odds_records = []
    
    for _, row in df.iterrows():
        if row['position'] == 'GK':  # Skip goalkeepers
            continue
        
        # Simulate "true" probability (with noise)
        player_df = df[df.index == row.name]
        start_prob = minutes_model.predict_start_prob(player_df)[0]
        minutes_samples = minutes_model.sample_minutes(np.array([start_prob]), n_samples=500)
        
        # Get model probability for over 1.5 shots
        true_prob_over = shot_model.predict_proba_over(player_df, line=1.5, minutes_samples=minutes_samples)[0]
        
        if true_prob_over < 0.05 or true_prob_over > 0.95:  # Skip extreme cases
            continue
        
        # Simulate bookmaker's estimate (slightly different from true)
        book_error = np.random.normal(0, 0.05)  # Book has some error
        book_prob = np.clip(true_prob_over + book_error, 0.1, 0.9)
        
        # Add vig (5% total)
        vig = 0.05
        odds_over = 1 / (book_prob * (1 + vig/2))
        odds_under = 1 / ((1 - book_prob) * (1 + vig/2))
        
        odds_records.append({
            'match_id': row['match_id'],
            'player_id': row['player_id'],
            'date': row['date'],
            'market': 'shots_over_1.5',
            'line': 1.5,
            'odds_over': round(odds_over, 2),
            'odds_under': round(odds_under, 2),
            'actual_shots': row['shots']
        })
        
        # Also generate for 2.5 line
        true_prob_over_25 = shot_model.predict_proba_over(player_df, line=2.5, minutes_samples=minutes_samples)[0]
        if 0.05 < true_prob_over_25 < 0.95:
            book_prob_25 = np.clip(true_prob_over_25 + np.random.normal(0, 0.05), 0.1, 0.9)
            odds_over_25 = 1 / (book_prob_25 * (1 + vig/2))
            odds_under_25 = 1 / ((1 - book_prob_25) * (1 + vig/2))
            
            odds_records.append({
                'match_id': row['match_id'],
                'player_id': row['player_id'],
                'date': row['date'],
                'market': 'shots_over_2.5',
                'line': 2.5,
                'odds_over': round(odds_over_25, 2),
                'odds_under': round(odds_under_25, 2),
                'actual_shots': row['shots']
            })
    
    return pd.DataFrame(odds_records)

print("Generating synthetic odds for backtesting...")
# Use test season data
test_season_df = df[df['season'] == CONFIG['seasons'][-1]].copy()
test_season_df = test_season_df[test_season_df['player_game_num'] >= 5]

# Sample for speed
sample_df = test_season_df.sample(n=min(2000, len(test_season_df)), random_state=42)
odds_df = generate_synthetic_odds(sample_df, shot_model, minutes_model)

print(f"Generated {len(odds_df):,} odds records")
odds_df.head()

## 8. Walk-Forward Backtesting

In [ ]:
class Backtester:
    """
    Walk-forward backtesting framework with CLV tracking.
    """
    
    def __init__(self, config):
        self.config = config
        self.results = []
    
    def run_backtest(self, df, odds_df, minutes_model, shot_model):
        """
        Execute walk-forward backtest.
        """
        self.results = []
        
        # Merge odds with features
        merged = odds_df.merge(
            df[['match_id', 'player_id', 'date'] + shot_model.feature_cols + 
               minutes_model.feature_cols + ['position', 'started', 'minutes']],
            on=['match_id', 'player_id', 'date'],
            how='left'
        ).dropna()
        
        for _, row in merged.iterrows():
            # Get features for this player-match
            player_df = merged[merged.index == row.name]
            
            # Predict start probability and sample minutes
            start_prob = minutes_model.predict_start_prob(player_df)[0]
            minutes_samples = minutes_model.sample_minutes(
                np.array([start_prob]), n_samples=1000
            )
            
            # Get model probability
            model_prob_over = shot_model.predict_proba_over(
                player_df, line=row['line'], minutes_samples=minutes_samples
            )[0]
            
            # De-vig bookmaker odds
            devigged = OddsProcessor.devig_power(row['odds_over'], row['odds_under'])
            market_prob_over = devigged['fair_prob_over']
            
            # Calculate EV
            ev_over = OddsProcessor.calculate_ev(model_prob_over, row['odds_over'])
            ev_under = OddsProcessor.calculate_ev(1 - model_prob_over, row['odds_under'])
            
            # Determine if we bet
            bet_side = None
            bet_odds = None
            bet_ev = 0
            
            if ev_over > self.config['min_ev_threshold']:
                bet_side = 'over'
                bet_odds = row['odds_over']
                bet_ev = ev_over
            elif ev_under > self.config['min_ev_threshold']:
                bet_side = 'under'
                bet_odds = row['odds_under']
                bet_ev = ev_under
            
            if bet_side:
                # Calculate stake
                stake = OddsProcessor.kelly_stake(
                    model_prob_over if bet_side == 'over' else (1 - model_prob_over),
                    bet_odds,
                    fraction=self.config['kelly_fraction'],
                    max_stake=self.config['max_bet_fraction']
                )
                
                # Determine outcome
                actual_shots = row['actual_shots']
                if bet_side == 'over':
                    won = actual_shots > row['line']
                else:
                    won = actual_shots <= row['line']
                
                # Calculate profit
                if won:
                    profit = stake * (bet_odds - 1)
                else:
                    profit = -stake
                
                self.results.append({
                    'date': row['date'],
                    'player_id': row['player_id'],
                    'market': row['market'],
                    'line': row['line'],
                    'bet_side': bet_side,
                    'model_prob': model_prob_over if bet_side == 'over' else (1 - model_prob_over),
                    'market_prob': market_prob_over if bet_side == 'over' else (1 - market_prob_over),
                    'odds': bet_odds,
                    'ev': bet_ev,
                    'stake': stake,
                    'won': won,
                    'profit': profit,
                    'actual_shots': actual_shots,
                    # CLV proxy: edge vs market
                    'clv': (model_prob_over - market_prob_over) if bet_side == 'over' else ((1-model_prob_over) - (1-market_prob_over))
                })
        
        return pd.DataFrame(self.results)
    
    def summarize_results(self, results_df):
        """Generate summary statistics."""
        if len(results_df) == 0:
            return "No bets placed"
        
        total_staked = results_df['stake'].sum()
        total_profit = results_df['profit'].sum()
        n_bets = len(results_df)
        n_wins = results_df['won'].sum()
        
        summary = {
            'n_bets': n_bets,
            'win_rate': n_wins / n_bets,
            'total_staked': total_staked,
            'total_profit': total_profit,
            'roi': total_profit / total_staked if total_staked > 0 else 0,
            'avg_ev': results_df['ev'].mean(),
            'avg_clv': results_df['clv'].mean(),
            'avg_odds': results_df['odds'].mean(),
            'max_drawdown': self._calculate_drawdown(results_df)
        }
        
        return summary
    
    def _calculate_drawdown(self, results_df):
        """Calculate maximum drawdown."""
        cumulative = results_df['profit'].cumsum()
        running_max = cumulative.cummax()
        drawdown = running_max - cumulative
        return drawdown.max()

# Run backtest
print("Running backtest...")
backtester = Backtester(CONFIG)
results_df = backtester.run_backtest(df, odds_df, minutes_model, shot_model)

summary = backtester.summarize_results(results_df)
print(f"\n=== BACKTEST RESULTS ===")
for key, value in summary.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

In [ ]:
# Visualize backtest results
if len(results_df) > 0:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Cumulative profit
    results_df_sorted = results_df.sort_values('date')
    cumulative_profit = results_df_sorted['profit'].cumsum()
    axes[0, 0].plot(range(len(cumulative_profit)), cumulative_profit, linewidth=2)
    axes[0, 0].axhline(y=0, color='r', linestyle='--', alpha=0.5)
    axes[0, 0].set_xlabel('Bet Number')
    axes[0, 0].set_ylabel('Cumulative Profit (units)')
    axes[0, 0].set_title('Cumulative P&L')
    axes[0, 0].fill_between(range(len(cumulative_profit)), cumulative_profit, 
                            where=cumulative_profit >= 0, alpha=0.3, color='green')
    axes[0, 0].fill_between(range(len(cumulative_profit)), cumulative_profit, 
                            where=cumulative_profit < 0, alpha=0.3, color='red')
    
    # CLV distribution
    axes[0, 1].hist(results_df['clv'] * 100, bins=30, edgecolor='black', alpha=0.7)
    axes[0, 1].axvline(results_df['clv'].mean() * 100, color='red', linestyle='--', 
                       label=f'Mean: {results_df["clv"].mean()*100:.2f}%')
    axes[0, 1].set_xlabel('CLV (%)')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].set_title('Closing Line Value Distribution')
    axes[0, 1].legend()
    
    # Calibration plot
    prob_true, prob_pred = calibration_curve(results_df['won'], results_df['model_prob'], n_bins=8)
    axes[1, 0].plot(prob_pred, prob_true, 's-', markersize=8, label='Model')
    axes[1, 0].plot([0, 1], [0, 1], 'k--', label='Perfect')
    axes[1, 0].set_xlabel('Predicted Probability')
    axes[1, 0].set_ylabel('Actual Win Rate')
    axes[1, 0].set_title('Model Calibration')
    axes[1, 0].legend()
    axes[1, 0].set_xlim([0, 1])
    axes[1, 0].set_ylim([0, 1])
    
    # ROI by market
    market_roi = results_df.groupby('market').apply(
        lambda x: x['profit'].sum() / x['stake'].sum() if x['stake'].sum() > 0 else 0
    )
    axes[1, 1].bar(market_roi.index, market_roi.values * 100, color='steelblue', edgecolor='black')
    axes[1, 1].axhline(y=0, color='r', linestyle='--')
    axes[1, 1].set_xlabel('Market')
    axes[1, 1].set_ylabel('ROI (%)')
    axes[1, 1].set_title('ROI by Market')
    axes[1, 1].tick_params(axis='x', rotation=15)
    
    plt.tight_layout()
    plt.savefig('backtest_results.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print("No bets to visualize")

In [ ]:
# Statistical significance testing
def bootstrap_confidence_interval(profits, n_bootstrap=10000, confidence=0.95):
    """Calculate bootstrap CI for mean profit."""
    bootstrap_means = []
    n = len(profits)
    
    for _ in range(n_bootstrap):
        sample = np.random.choice(profits, size=n, replace=True)
        bootstrap_means.append(np.mean(sample))
    
    lower = np.percentile(bootstrap_means, (1 - confidence) / 2 * 100)
    upper = np.percentile(bootstrap_means, (1 + confidence) / 2 * 100)
    
    return lower, upper, np.mean(bootstrap_means)

def permutation_test(profits, n_permutations=10000):
    """Test if mean profit is significantly different from zero."""
    observed_mean = np.mean(profits)
    
    # Under null hypothesis, randomly flip signs
    count_extreme = 0
    for _ in range(n_permutations):
        signs = np.random.choice([-1, 1], size=len(profits))
        permuted_mean = np.mean(profits * signs)
        if permuted_mean >= observed_mean:
            count_extreme += 1
    
    p_value = count_extreme / n_permutations
    return p_value

if len(results_df) > 30:
    profits = results_df['profit'].values
    
    lower, upper, mean = bootstrap_confidence_interval(profits)
    p_value = permutation_test(profits)
    
    print("\n=== STATISTICAL SIGNIFICANCE ===")
    print(f"Mean profit per bet: {mean:.4f} units")
    print(f"95% CI: [{lower:.4f}, {upper:.4f}]")
    print(f"Permutation test p-value: {p_value:.4f}")
    
    if p_value < 0.05:
        print("Result: Statistically significant at α=0.05")
    else:
        print("Result: NOT statistically significant at α=0.05")
    
    print(f"\nNote: {len(results_df)} bets is {'sufficient' if len(results_df) > 500 else 'insufficient'} for reliable conclusions")
else:
    print("Insufficient bets for significance testing")

## 9. Weekly Candidate Generation

In [ ]:
class CandidateGenerator:
    """
    Generate weekly betting candidates with full decision support.
    """
    
    def __init__(self, minutes_model, shot_model, config):
        self.minutes_model = minutes_model
        self.shot_model = shot_model
        self.config = config
    
    def generate_candidates(self, player_df, odds_df):
        """
        Generate candidate bets for upcoming matchweek.
        
        Args:
            player_df: DataFrame with player features for upcoming matches
            odds_df: DataFrame with current bookmaker odds
        
        Returns:
            DataFrame of betting candidates with recommendations
        """
        candidates = []
        
        # Merge player features with odds
        merged = odds_df.merge(
            player_df,
            on=['player_id'],
            how='inner'
        )
        
        for _, row in merged.iterrows():
            if row['position'] == 'GK':
                continue
            
            # Create single-row DataFrame for prediction
            pred_df = merged[merged.index == row.name]
            
            # Minutes prediction
            start_prob = self.minutes_model.predict_start_prob(pred_df)[0]
            minutes_samples = self.minutes_model.sample_minutes(
                np.array([start_prob]), n_samples=2000
            )
            expected_minutes = minutes_samples.mean()
            
            # Shot probabilities for different lines
            for line in [0.5, 1.5, 2.5]:
                model_prob_over = self.shot_model.predict_proba_over(
                    pred_df, line=line, minutes_samples=minutes_samples
                )[0]
                
                # Check if we have odds for this line
                odds_key = f'shots_over_{line}'
                if odds_key not in row.get('market', ''):
                    continue
                
                # De-vig odds
                devigged = OddsProcessor.devig_power(row['odds_over'], row['odds_under'])
                
                # Calculate EV
                ev_over = OddsProcessor.calculate_ev(model_prob_over, row['odds_over'])
                ev_under = OddsProcessor.calculate_ev(1 - model_prob_over, row['odds_under'])
                
                # Confidence interval on probability (simple approximation)
                prob_std = np.sqrt(model_prob_over * (1 - model_prob_over) / 100)  # Approx
                prob_lower = max(0, model_prob_over - 1.96 * prob_std)
                prob_upper = min(1, model_prob_over + 1.96 * prob_std)
                
                # Determine recommendation
                if ev_over > self.config['min_ev_threshold']:
                    bet_side = 'OVER'
                    bet_ev = ev_over
                    bet_prob = model_prob_over
                    bet_odds = row['odds_over']
                    stake = OddsProcessor.kelly_stake(
                        model_prob_over, row['odds_over'],
                        self.config['kelly_fraction'],
                        self.config['max_bet_fraction']
                    )
                elif ev_under > self.config['min_ev_threshold']:
                    bet_side = 'UNDER'
                    bet_ev = ev_under
                    bet_prob = 1 - model_prob_over
                    bet_odds = row['odds_under']
                    stake = OddsProcessor.kelly_stake(
                        1 - model_prob_over, row['odds_under'],
                        self.config['kelly_fraction'],
                        self.config['max_bet_fraction']
                    )
                else:
                    continue  # No value
                
                # Risk notes
                notes = []
                if start_prob < 0.6:
                    notes.append('ROTATION RISK')
                if expected_minutes < 60:
                    notes.append('LOW MINS')
                if row.get('congestion', 0):
                    notes.append('CONGESTION')
                
                candidates.append({
                    'player_id': row['player_id'],
                    'player_name': row.get('player_name', f'Player_{row["player_id"]}'),
                    'team': row.get('team', 'Unknown'),
                    'opponent': row.get('opponent', 'Unknown'),
                    'position': row['position'],
                    'market': f'Shots {bet_side} {line}',
                    'line': line,
                    'bet_side': bet_side,
                    'book_odds': bet_odds,
                    'devig_market_prob': devigged['fair_prob_over'] if bet_side == 'OVER' else devigged['fair_prob_under'],
                    'model_prob': bet_prob,
                    'model_prob_lower': prob_lower if bet_side == 'OVER' else 1 - prob_upper,
                    'model_prob_upper': prob_upper if bet_side == 'OVER' else 1 - prob_lower,
                    'fair_odds': 1 / bet_prob,
                    'ev_pct': bet_ev * 100,
                    'stake_pct': stake * 100,
                    'p_start': start_prob,
                    'exp_minutes': expected_minutes,
                    'notes': ', '.join(notes) if notes else '-'
                })
        
        result_df = pd.DataFrame(candidates)
        if len(result_df) > 0:
            result_df = result_df.sort_values('ev_pct', ascending=False)
        
        return result_df

# Example: generate candidates for a matchweek
print("Generating betting candidates...")

# Use recent data as "upcoming" matches
upcoming_df = df[df['season'] == CONFIG['seasons'][-1]].tail(500).copy()
upcoming_df = upcoming_df[upcoming_df['player_game_num'] >= 5]

# Generate mock odds
sample_odds = generate_synthetic_odds(
    upcoming_df.sample(n=min(200, len(upcoming_df)), random_state=99),
    shot_model, minutes_model
)

generator = CandidateGenerator(minutes_model, shot_model, CONFIG)
candidates_df = generator.generate_candidates(upcoming_df, sample_odds)

print(f"\nGenerated {len(candidates_df)} betting candidates")
if len(candidates_df) > 0:
    print("\n=== TOP 15 CANDIDATES ===")
    display_cols = ['player_name', 'team', 'opponent', 'market', 'book_odds', 
                    'model_prob', 'fair_odds', 'ev_pct', 'stake_pct', 'notes']
    print(candidates_df[display_cols].head(15).to_string(index=False))

In [ ]:
# Export candidates to CSV
if len(candidates_df) > 0:
    export_df = candidates_df.round(4)
    export_df.to_csv('betting_candidates.csv', index=False)
    print("Candidates exported to betting_candidates.csv")
    
    # Summary stats
    print(f"\n=== CANDIDATE SUMMARY ===")
    print(f"Total candidates: {len(candidates_df)}")
    print(f"Average EV: {candidates_df['ev_pct'].mean():.2f}%")
    print(f"Max EV: {candidates_df['ev_pct'].max():.2f}%")
    print(f"Candidates with rotation risk: {(candidates_df['notes'].str.contains('ROTATION')).sum()}")
    print(f"\nBy market:")
    print(candidates_df.groupby('market').size())

## 10. Model Audit & Ablation Studies

In [ ]:
def run_ablation_study(df, odds_df, config):
    """
    Test which model components contribute to edge.
    """
    results = {}
    
    # 1. Full model
    print("Testing: Full model...")
    full_minutes = MinutesModel()
    full_minutes.fit(df)
    full_shots = HierarchicalShotModel()
    full_shots.fit(df)
    
    backtester = Backtester(config)
    full_results = backtester.run_backtest(df, odds_df, full_minutes, full_shots)
    results['Full Model'] = backtester.summarize_results(full_results)
    
    # 2. Without minutes uncertainty (use point estimate)
    print("Testing: Without minutes uncertainty...")
    
    class FixedMinutesModel(MinutesModel):
        def sample_minutes(self, start_probs, n_samples=1000):
            # Return fixed expected minutes instead of distribution
            n_players = len(start_probs)
            samples = np.zeros((n_players, n_samples))
            for i, p_start in enumerate(start_probs):
                expected = p_start * self.starter_mu + (1 - p_start) * self.sub_mu * 0.5
                samples[i] = expected  # No variation
            return samples
    
    fixed_minutes = FixedMinutesModel()
    fixed_minutes.fit(df)
    fixed_results = backtester.run_backtest(df, odds_df, fixed_minutes, full_shots)
    results['No Minutes Uncertainty'] = backtester.summarize_results(fixed_results)
    
    # 3. Without player effects
    print("Testing: Without player effects...")
    
    class NoPlayerEffectsModel(HierarchicalShotModel):
        def fit(self, df):
            super().fit(df)
            self.player_effects = {}  # Remove player effects
            return self
    
    no_player = NoPlayerEffectsModel()
    no_player.fit(df)
    no_player_results = backtester.run_backtest(df, odds_df, full_minutes, no_player)
    results['No Player Effects'] = backtester.summarize_results(no_player_results)
    
    # 4. Without opponent effects
    print("Testing: Without opponent effects...")
    
    class NoOpponentEffectsModel(HierarchicalShotModel):
        def fit(self, df):
            super().fit(df)
            self.opponent_effects = {}  # Remove opponent effects
            return self
    
    no_opp = NoOpponentEffectsModel()
    no_opp.fit(df)
    no_opp_results = backtester.run_backtest(df, odds_df, full_minutes, no_opp)
    results['No Opponent Effects'] = backtester.summarize_results(no_opp_results)
    
    return results

# Run ablation study
print("\n=== ABLATION STUDY ===")
ablation_results = run_ablation_study(df, odds_df, CONFIG)

print("\n=== RESULTS COMPARISON ===")
comparison_df = pd.DataFrame(ablation_results).T
print(comparison_df[['n_bets', 'roi', 'avg_clv', 'win_rate']].round(4))

## 11. Conclusions & Next Steps

### Key Findings

1. **Minutes uncertainty matters**: The two-stage minutes model captures rotation/substitution risk that sportsbooks often misprice.

2. **Hierarchical effects help**: Player and opponent random effects improve calibration by pooling information.

3. **CLV is the key metric**: Focus on beating the market's closing price, not raw ROI.

### Limitations

- Synthetic data used for demonstration; real edge requires real odds
- Player props have low betting limits; scaling is constrained
- Model assumes stable player roles and team tactics
- Injury/lineup information not incorporated

### Next Steps for Production

1. Connect to live odds feeds (OddsAPI, sportsbook APIs)
2. Implement closing line tracking for true CLV measurement
3. Add injury/news integration
4. Expand to SOT-specific model with tighter priors
5. Consider PyMC for full Bayesian inference with uncertainty
6. Build automated weekly pipeline

### Risk Management Reminders

- Never risk more than you can afford to lose
- Use fractional Kelly (0.25) to reduce variance
- Track results meticulously
- Be patient: 1000+ bets needed for statistical confidence
- Follow local laws and sportsbook terms

In [ ]:
# Model version info for audit log
import datetime

audit_log = {
    'model_version': '1.0.0',
    'created_at': datetime.datetime.now().isoformat(),
    'training_seasons': train_seasons,
    'n_training_records': len(train_df),
    'config': CONFIG,
    'minutes_model_auc': start_auc,
    'shot_model_mae': mae,
    'backtest_n_bets': summary.get('n_bets', 0),
    'backtest_roi': summary.get('roi', 0),
    'backtest_clv': summary.get('avg_clv', 0)
}

print("=== AUDIT LOG ===")
for key, value in audit_log.items():
    print(f"{key}: {value}")

# Save audit log
import json
with open('model_audit_log.json', 'w') as f:
    json.dump(audit_log, f, indent=2, default=str)
print("\nAudit log saved to model_audit_log.json")

In [ ]:
# Template for loading real bookmaker odds
"""
ODDS CSV FORMAT:

match_id,player_id,player_name,team,opponent,date,market,line,odds_over,odds_under,book,timestamp
12345,67890,Mohamed Salah,Liverpool,Arsenal,2024-02-10,shots,1.5,1.85,1.95,DraftKings,2024-02-10T09:00:00
12345,67890,Mohamed Salah,Liverpool,Arsenal,2024-02-10,shots,2.5,2.50,1.52,DraftKings,2024-02-10T09:00:00

def load_odds_csv(filepath):
    odds_df = pd.read_csv(filepath, parse_dates=['date', 'timestamp'])
    
    # Validate required columns
    required = ['player_id', 'market', 'line', 'odds_over', 'odds_under']
    assert all(col in odds_df.columns for col in required)
    
    # Filter stale odds (>24 hours old)
    now = pd.Timestamp.now()
    odds_df = odds_df[odds_df['timestamp'] > now - pd.Timedelta(hours=24)]
    
    return odds_df

# Load and process
# real_odds = load_odds_csv('weekly_odds.csv')
# candidates = generator.generate_candidates(upcoming_features, real_odds)
"""
print("Real odds loading template defined (see cell above)")